## Обработка Словаря заимствований китайского языка и данных БКРС

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import html
import urllib
import time
import re
import textdistance

### Часть 1. Вспомогательный код для анализа Словаря заимствований китайского языка

In [197]:
df_phon = pd.read_csv('wlc_cd.csv', delimiter=',', index_col='id')
#df_phon = df_phon.drop(['pinyin_1', 'page'], 1)
df_phon.head()

,word,pinyin_final,origin,lang,Unnamed: 6,rus_transcription
id,,,,,,
0,阿尔申,āěrshén,аршин,NaN,NaN,аршын
1,阿尔西非合金,āěrxǐfēi,альсифер,NaN,NaN,ал’с’иф’эр
2,阿法林,āfǎlín,афалина,NaN,NaN,афал’ина
3,阿留米特,āliúmǐtè,алюмит,NaN,NaN,ал’ум’ит
4,阿依尔,āyi'ěr,аил,NaN,NaN,аил


In [10]:
codas = '[аоуыэи]'
coda_positions = '([йцкншщзхфвпрлджчсмтб]|$)'

In [318]:
hard_codas = '[аоуыэ]'
coda_position = '[йцкншщзхъфвпрлджчсмтб$]'
soft_codas = '[яёиюе]'
yotated = '[яёюе]'
yot_position = '[ьаоуыэяёюие]'
all_codas = '[яёюеаоуыэ]'

In [5]:
plosives = '[бпгк]'
plosives_soft = '[]'
fricatives = '[вфзсшжх]'
affricates = '[цч дж]'
semi_affricates = '[тд]'
nasals = '[мн]'
liquids = '[рл]'
glides = 'й'

In [3]:
def subsetting(onset, coda, soft=False):
    string = onset+coda
    if soft == True:
        string = onset + '’' + coda
    else:
        string = onset + coda
    subset = df_phon[df_phon['rus_transcription'].str.contains(string)]
    return subset

In [92]:
subsetting('л', coda_positions, soft=True)

C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  import sys


,word,pinyin_final,origin,lang,Unnamed: 6,rus_transcription
id,,,,,,
1,阿尔西非合金,āěrxǐfēi,альсифер,NaN,NaN,ал’с’иф’эр
15,奥克索尔油,àokèsuǒ'ěr(yóu),оксоль,NaN,NaN,аксол’
33,比奥熙诺尔,bǐ'àoxīnuò'ěr,биохиноль,NaN,NaN,б’иох’инол’
51,布尔什维克,bù'ěrshíwéikè,большевик,NaN,NaN,бал’шыв’ик
73,杜阿登诺尔布,dù'ādēngnuò'ěr(bù),туальденор,NaN,NaN,туал’дынор
91,伏尔泰椅,fú'ěrtài(yǐ),вольтеровское кресло,NaN,NaN,вал’тэр
103,格兰尼托皮,gélánnítuōpí,гранитоль,NaN,NaN,гран’итол’
107,格丝里,gésīlǐ,гесль,NaN,NaN,г’эсл’
132,吉协力,jíxiélì,кисель,NaN,NaN,к’ис’эл’


### Часть 2. Анализ данных из словаря БКРС

In [2]:
df_bkrs = pd.read_csv('data_with_translit.csv', delimiter=';')
df_bkrs['translit'] = df_bkrs['translit'].replace(to_replace=r' ', value=r'', regex=True)
df_bkrs['translit'] = df_bkrs['translit'].replace(to_replace=r'’', value=r'', regex=True)
df_bkrs = df_bkrs.drop_duplicates()
df_bkrs['label_ru'] = df_bkrs['label_ru'].str.lower()
df_bkrs.head(20)

,label_zh,label_ru,translit
0,奥涅加,онежское,àonièjiā
1,奥特拉德诺耶,отрадное,àotèlādénuòyē
2,贝加尔,байкал,bèijiāěr
3,拉多加,ладожское,lāduōjiā
4,伏尔加,волго,fúěrjiā
5,普斯科夫,псковское,pǔsīkēfū
6,列佩利,лепельское,lièpèilì
7,哈桑,хасан,hāsāng
8,谢利格尔,селигер,xièlìgéěr
9,拉马,лама,lāmǎ


In [4]:
df_bkrs.shape

(1495, 3)

In [3]:
def subsetting_bkrs_3(onset, coda, analogue=None):
    string = onset+coda
    subset = df_bkrs[df_bkrs['label_ru'].str.contains(string)] #df_bkrs[df_bkrs
    if analogue is not None:
        counter = 0
        #subset = subset[subset['translit'].str.contains(analogue)]
        for index, row in subset.iterrows():
            word_padding = '__' + row.translit + '__' #row.pinyin_final
            subset_indices = re.finditer(string, row.label_ru) #origin
            for s_i in subset_indices:
                l_index = int(s_i.span()[0])
                print(word_padding)
                print('expecting symbol on '+ str(l_index))
                #span: +- 2 symbols to both sides
                if len(analogue) == 1:
                    span_l, span_r = l_index, l_index+4
                elif len(analogue) == 2:
                    span_l, span_r = l_index, l_index+5
                print(word_padding[span_l:span_r], analogue)
                if analogue in word_padding[span_l:span_r]:
                    if len(analogue) == 1 and analogue in ['s', 'z', 'c', 'n']:
                        if analogue == 'n':
                            t = analogue + 'g'
                        else:
                            t = analogue+'h'
                        if t in word_padding[span_l:span_r+1]:
                            print('false positive')
                        else:
                            print('FP avoided')
                            print(word_padding)
                            print(row)
                            counter += 1
                    else:
                        print(l_index)
                        print(word_padding)
                        print(row)
                        counter += 1
    return counter

In [4]:
codas_hard = '[аоуыэ]'
codas_soft = '[яёюие]' 
coda_positions_contrastive = '[рлмн]'
coda_positions_voc_no_contrast = '[бдгзвж]'
coda_positions_sil_no_contrast = '([цкшхфпст]|$)'
coda_positions_soft = '([чщйь]|[бтсщзфвд][яёюиеь])'
coda_position_no_contrast = '([йцкнгшщзхфвпрлджчсмтб ]|$)'
yotated = '[яёюе]'
yot_position = '(\A|[ьаоуыэяёюие])'
unpaired_codas = '[аоуыэяёюие]'
unpaired_coda_positions = 'ь'

In [27]:
subsetting_bkrs_3('х', coda_positions_soft, analogue='f')

__shāhèqiáoěrsīkè__
expecting symbol on 2
shāh f
__bāhèqísàlài__
expecting symbol on 2
bāhè f
__qíhèwén__
expecting symbol on 2
qíhè f
__bāhèqísàlài__
expecting symbol on 2
bāhè f
__jìhèwēn__
expecting symbol on 2
jìhè f
__bōhèwéisītènièwò__
expecting symbol on 2
bōhè f
__lāhèjiānbōxīyà__
expecting symbol on 2
lāhè f
__lāhèjiānbōxīyà__
expecting symbol on 8
ānbō f
__kǎmiǎnsīkèshāhèjīngsījī__
expecting symbol on 10
kèsh f


C:\Users\lenovo\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  This is separate from the ipykernel package so we can avoid doing imports until


0